In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import warnings
import json
import re
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

In [129]:
json_file = 'cities.json'

with open(json_file, 'r') as f:
    md = json.load(f)

In [106]:
md

{'Albuquerque city': {'name': 'Albuquerque',
  'odp': 'N',
  'odp_launch': '',
  'portal': 'Y',
  'portal_launch': '01-01-2012',
  'portal_type': 'NR',
  'robust_portal': 'Y',
  'state': 'NM',
  'wwc': 'Y'},
 'Alexandria city': {'name': 'Alexandria',
  'odp': 'N',
  'odp_launch': '',
  'portal': 'Y',
  'portal_launch': '1-1-2000',
  'portal_type': 'GQ',
  'robust_portal': 'N',
  'state': 'VA',
  'wwc': 'N'},
 'Arlington city': {'name': 'Arlington',
  'odp': 'N',
  'odp_launch': '',
  'portal': 'N',
  'portal_launch': '',
  'portal_type': 'GQ',
  'state': 'WA',
  'wwc': 'N'},
 'Asheville city': {'name': 'Asheville',
  'odp': 'Y',
  'odp_launch': '10-13-2015',
  'portal': 'Y',
  'portal_launch': '06-01-2016',
  'portal_type': 'C',
  'robust_portal': 'Y',
  'state': 'NC',
  'wwc': 'N'},
 'Bainbridge Island city': {'name': 'Bainbridge',
  'odp': 'N',
  'odp_launch': '',
  'portal': 'N',
  'portal_launch': '',
  'portal_type': 'NR',
  'state': 'WA',
  'wwc': 'N'},
 'Belleville city': {'name

In [153]:
#need to add info for non-WWC cities

cities = pd.read_csv('../../../WWC_basics.csv')

In [154]:
cs = cities['City, State'].str.split(",", expand = True)
cities['city'] = cs[0] + ' city'

In [157]:
cities['State']

0      NM
1      AK
2      TX
3      GA
4      GA
5      MD
6      LA
7      WA
8      AL
9      ID
10     MA
11     CO
12     NY
13     MA
14     FL
15     NC
16     SC
17     NC
18     TN
19     CA
20     OH
21     CO
22     SC
23     OH
24     CA
25     TX
26     CO
27     IA
28     CA
29     NC
30     ND
31     NC
32     CO
33     FL
34     TX
35     FL
36     AZ
37     AZ
38     MI
39     NC
40     OR
41     CT
42     CA
43     HI
44     MO
45     IN
46     TX
47     MS
48     FL
49     KS
50     MO
51     TN
52     LA
53     TX
54     NV
55     TX
56     KY
57     NE
58     AR
59     CA
60     KY
61     WI
62     TX
63     TN
64     AZ
65     FL
66     WI
67     MN
68     CA
69     IL
70     TN
71     CT
72     LA
73     VA
74     OK
75     KS
76     FL
77     CA
78     TX
79     OR
80     RI
81     NC
82     CA
83     VA
84     CA
85     MN
86     CA
87     UT
88     CA
89     CA
90     AZ
91     WA
92     SD
93     IN
94     NY
95     WA
96     AZ
97     KS
98     OK
99     TX


In [158]:
cities = cities[['city', 'State', 'Pol_Lean_State', 'Region', 'Governance', 'Budget_1617', 'Exec_Party']]

In [8]:
df = pd.read_csv('/Users/alenastern/Google Drive File Stream/My Drive/Alena_Project/PR_Data/Riverside.csv')

In [10]:
k = 'Albuquerque city'

In [107]:

md = {"Arlington city": {
 	"name": "Arlington",
 	"state": "WA",
 	"wwc":"N",
 	"portal_type": "GQ",
	"odp":"N",
	"odp_launch":"",
	"portal": "N",
	"portal_launch":"" 
	}}

In [134]:
data = pd.DataFrame(columns = ['month_year', 'count', 'city', 'state', 'odp_date', 'policy', 'portal_date', 'portal', 
                              'year', 'months', '2015', '2016', '2017', '2018'])
for key, value in md.items():
    city = value['name']
    filepath = '/Users/alenastern/Google Drive File Stream/My Drive/Alena_Project/PR_Data/{}.csv'.format(city)
    try:
        df = pd.read_csv(filepath)
    except:
        try:
            df = pd.read_csv(filepath, encoding='mac_roman')
        except:
            #try:
            #    df = pd.read_csv(filepath, engine = 'python')
            #except:
            continue
    print(key)
    df['Create Date'] = pd.to_datetime(df['Create Date'])
    df['month_year'] = df['Create Date'].dt.to_period('M')
    month_count = df.groupby('month_year').count()
    month_count.rename(index=str, columns={"Create Date": "count"}, inplace = True)
    month_count.reset_index(inplace=True)
    

    mc = month_count[['month_year', 'count']]
    #remove July data
    mc = mc[(mc['month_year'] != '07-2018') & (mc['month_year'] != '2018-07') ] 
    mc['city'] = key
    mc['state'] = value['state']
    if value['odp'] == "Y":
        odp_dl = value['odp_launch'].split('-')
        mc['odp_date'] = dt.datetime(int(odp_dl[2]),int(odp_dl[0]),int(odp_dl[1]))
        mc['policy'] = np.where(pd.to_datetime(mc['month_year'])< mc['odp_date'], 0, 1)
    else:
        mc['odp_date'] = 'NaN'
        mc['policy'] = 0
        
    if value['portal'] == "Y":
        portal_dl = value['portal_launch'].split('-')
        mc['portal_date'] = dt.datetime(int(portal_dl[2]),int(portal_dl[0]),int(portal_dl[1]))
        mc['portal'] = np.where(pd.to_datetime(mc['month_year'])< mc['portal_date'], 0, 1)
    else:
        mc['portal_date'] = 'NaN'
        mc['portal'] = 0
        
    mc['year'] = pd.to_datetime(mc['month_year']).dt.year
    
    first_month = min(df['month_year'])
    
    #remove first month of data
    mc = mc[mc['month_year'] != str(first_month)]
    
    mc['months'] = ((pd.to_datetime(mc['month_year']).dt.year - first_month.year) * 12 + 
            pd.to_datetime(mc['month_year']).dt.month - first_month.month)

    
    mc['2015'] = np.where(mc['year']== 2015, 1, 0)
    mc['2016'] = np.where(mc['year']== 2016, 1, 0)
    mc['2017'] = np.where(mc['year']== 2017, 1, 0)
    mc['2018'] = np.where(mc['year']== 2018, 1, 0)
    
    data = pd.concat([data, mc])
        
    

Albuquerque city
Alexandria city
Arlington city
Bainbridge Island city
Belleville city
Boulder city
Cape Coral city
Cathedral City city
Corona city
Dayton city
Denton city
El Dorado county
Everett city
Fort Collins city
Fort Worth city
Greensboro city
Hayward city
Joliet city
Kirkland city
Las Cruces city
Las Vegas city
Lynnwood city
Mercer Island city
Miami city
Middleborough city
New Orleans city
Oakland city
Oaklahoma City city
Olympia city
Palo Alto city
Peoria city
Providence city
Pullman city
Rancho Cucamonga city
Redmond city
Renton city
Riverside city
Sacramento city
Salt Lake City city
San Francisco city
San Mateo city
Tukwila city
Vallejo city
Washington city
West Sacramento city
Winchester city


In [135]:
data.shape

(1322, 14)

In [149]:
mc_mg.shape

(1350, 47)

In [133]:
len(data.city.unique())

46

In [58]:
#mc['time']= (pd.to_datetime(mc['month_year']).dt.year - mc['odp_date'].dt.year) * 12 + pd.to_datetime(mc['month_year']).dt.month - mc['odp_date'].dt.month

In [61]:
mc.head()

,month_year,count,city,state,odp_date,year,time,policy,treatment
0,2015-12,13,Riverside city,CA,2017-03-07,2015,-15,0,1
1,2016-01,47,Riverside city,CA,2017-03-07,2016,-14,0,1
2,2016-02,49,Riverside city,CA,2017-03-07,2016,-13,0,1
3,2016-03,43,Riverside city,CA,2017-03-07,2016,-12,0,1
4,2016-04,39,Riverside city,CA,2017-03-07,2016,-11,0,1


In [136]:
cd = pd.read_csv('city_data.csv')

In [19]:
cd.head()

,NAME,population,total_hh,pct_25_34,pct_hispanic,pct_white,pct_black,median_age,gross_rent,median_hh_income,pct_bachelor,state,place,year,avg_hh_size,city,state_name
0,"Birmingham city, Alabama",213434,90117.0,17.2,3.6,24.6,71.3,36.0,777,36241,16.9,1,7000,2016,2.368410,Birmingham city,Alabama
1,"Dothan city, Alabama",67714,26119.0,13.6,2.9,60.9,35.0,38.9,720,42336,14.7,1,21184,2016,2.592519,Dothan city,Alabama
2,"Hoover city, Alabama",84943,32110.0,15.3,7.6,70.9,16.7,36.6,1021,79004,34.1,1,35896,2016,2.645375,Hoover city,Alabama
3,"Huntsville city, Alabama",196225,80540.0,14.3,6.3,60.6,31.5,35.6,766,51895,25.7,1,37000,2016,2.436367,Huntsville city,Alabama
4,"Mobile city, Alabama",192895,79188.0,15.8,2.5,46.4,49.8,36.9,804,39463,18.9,1,50000,2016,2.435912,Mobile city,Alabama


In [148]:
 data.groupby('city').count()

,month_year,count,state,odp_date,policy,portal_date,portal,year,months,2015,2016,2017,2018,city_state
city,,,,,,,,,,,,,,
Albuquerque city,35,35,35,35,35,35,35,35,35,35,35,35,35,35
Alexandria city,95,95,95,95,95,95,95,95,95,95,95,95,95,95
Arlington city,28,28,28,28,28,28,28,28,28,28,28,28,28,28
Bainbridge Island city,26,26,26,26,26,26,26,26,26,26,26,26,26,26
Belleville city,65,65,65,65,65,65,65,65,65,65,65,65,65,65
Boulder city,9,9,9,9,9,9,9,9,9,9,9,9,9,9
Cape Coral city,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Cathedral City city,10,10,10,10,10,10,10,10,10,10,10,10,10,10
Corona city,29,29,29,29,29,29,29,29,29,29,29,29,29,29


In [137]:
data['city_state'] = data['city'] + data['state']
cd['city_state'] = cd['city'] + cd['state_code']
cd_mg = data.merge(cd, how ='left', on = 'city_state')
cd_mg.rename(index=str, columns={'city_x': 'city'}, inplace = True)

In [160]:
cities['city_state'] = cities['city'] + cities['State']

In [161]:
mc_mg = cd_mg.merge(cities, how ='left', on = 'city_state')

In [162]:
mc_mg.shape

(1322, 48)

In [100]:
# create year dummies
#mc_mg['2013'] = np.where(mc_mg['year_x']== 2013, 1, 0)
#mc_mg['2014'] = np.where(mc_mg['year_x']== 2014, 1, 0)
mc_mg['2015'] = np.where(mc_mg['year_x']== 2015, 1, 0)
mc_mg['2016'] = np.where(mc_mg['year_x']== 2016, 1, 0)
mc_mg['2017'] = np.where(mc_mg['year_x']== 2017, 1, 0)
mc_mg['2018'] = np.where(mc_mg['year_x']== 2018, 1, 0)

In [101]:
# create time since policy dummies
#mc_mg['0_6mos_odp'] = np.where((mc_mg['time']>= 0) & (mc_mg['time']<6), 1, 0)
#mc_mg['6_12mos_odp'] = np.where((mc_mg['time']>= 6) & (mc_mg['time']<12), 1, 0)
#mc_mg['12_18mos_odp'] = np.where((mc_mg['time']>= 12) & (mc_mg['time']<18), 1, 0)
#mc_mg['18_24mos_odp'] = np.where((mc_mg['time']>= 18) & (mc_mg['time']<24), 1, 0)
#mc_mg['over_24mos_odp'] = np.where(mc_mg['time']>= 24, 1, 0)

In [163]:
# create governance dummies
mc_mg['mayor_council'] = np.where(mc_mg['Governance']== 'Mayor-Council', 1, 0)
mc_mg['council_manager'] = np.where(mc_mg['Governance']== 'Council-Manager', 1, 0)
#mc_mg['mayor_commission'] = np.where(mc_mg['Governance']== 'Mayor-Commission', 1, 0)
#mc_mg['commissioner_manager'] = np.where(mc_mg['Governance']== 'Commissioner-Manager', 1, 0)
#mc_mg['other'] = np.where((mc_mg['Governance']!= 'Council-Manager') & (mc_mg['Governance']!= 'Mayor-Council'), 1, 0)

In [164]:
# create state political lean dummies
#mc_mg['competitive'] = np.where(mc_mg['Pol_Lean_State']== 'Competitive', 1, 0)
#mc_mg['lean_dem'] = np.where(mc_mg['Pol_Lean_State']== 'Lean Democratic', 1, 0)
#mc_mg['solid_dem'] = np.where(mc_mg['Pol_Lean_State']== 'Solid Democratic', 1, 0)
#mc_mg['lean_rep'] = np.where(mc_mg['Pol_Lean_State']== 'Lean Republican', 1, 0)
#mc_mg['solid_rep'] = np.where(mc_mg['Pol_Lean_State']== 'Solid Republicans', 1, 0)

mc_mg['rep'] = np.where((mc_mg['Pol_Lean_State']== 'Lean Republican') | (mc_mg['Pol_Lean_State']== 'Solid Republican'), 1, 0)
mc_mg['dem'] = np.where((mc_mg['Pol_Lean_State']== 'Lean Democrat') | (mc_mg['Pol_Lean_State']== 'Solid Democrat'), 1, 0)


In [165]:
# create interaction effect variables

mc_mg['pop_treat'] = mc_mg['population'] * mc_mg['policy']
mc_mg['age_treat'] = mc_mg['median_age'] * mc_mg['policy']
mc_mg['rent_treat'] = mc_mg['median_gross_rent'] * mc_mg['policy']
mc_mg['income_treat'] = mc_mg['median_income'] * mc_mg['policy']
mc_mg['25_treat'] = mc_mg['pct_25_34'] * mc_mg['policy']
mc_mg['male_treat'] = mc_mg['pct_male'] * mc_mg['policy']
mc_mg['black_treat'] = mc_mg['pct_black'] * mc_mg['policy']
mc_mg['bachelor_treat'] = mc_mg['pct_bachelor'] * mc_mg['policy']
mc_mg['budget_treat'] = mc_mg['Budget_1617'] * mc_mg['policy']
mc_mg['2015_treat'] = mc_mg['2015'] * mc_mg['policy']
mc_mg['2016_treat'] = mc_mg['2016'] * mc_mg['policy']
mc_mg['2017_treat'] = mc_mg['2017'] * mc_mg['policy']
mc_mg['2018_treat'] = mc_mg['2018'] * mc_mg['policy']
mc_mg['mayor_treat'] = mc_mg['mayor_council'] * mc_mg['policy']
mc_mg['manager_treat'] = mc_mg['council_manager'] * mc_mg['policy']
mc_mg['dem_treat'] = mc_mg['dem'] * mc_mg['policy']
mc_mg['rep_treat'] = mc_mg['rep'] * mc_mg['policy']
mc_mg['time_treat'] = mc_mg['months'] * mc_mg['policy']

In [167]:
clean_data = mc_mg[['month_year', 'count', 'city_x', 'state_x', 'months', 'policy', 'population',
                   'median_age', 'median_gross_rent', 'median_income', 'pct_25_34', 'pct_male',
                  'pct_black', 'pct_white', 'pct_bachelor', 'Budget_1617', '2015',
                  '2016', '2017', '2018','mayor_council', 'council_manager', 'dem', 'rep',
                   'pop_treat', 'age_treat', 'rent_treat', 'income_treat', '25_treat', 'male_treat',
                   'black_treat', 'bachelor_treat', 'budget_treat', '2015_treat', '2016_treat',
                   '2017_treat', '2018_treat', 'mayor_treat', 'manager_treat', 'dem_treat', 
                   'rep_treat', 'time_treat']]

In [168]:
clean_data.to_csv('clean_data.csv', index = False)

In [166]:
mc_mg[:100]

,month_year,count,city_x,state_x,odp_date,policy,portal_date,portal,year_x,months,2015,2016,2017,2018,city_state,NAME,population,m_25_29,m_30_34,f_25_29,f_30_34,total_male,total_white,total_black,total_hispanic,median_age,median_gross_rent,median_income,total_bachelor,state_y,place,year_y,pct_25_34,pct_male,pct_black,pct_white,pct_hispanic,pct_bachelor,city_y,state_name,state_code,city_y,State,Pol_Lean_State,Region,Governance,Budget_1617,Exec_Party,mayor_council,council_manager,rep,dem,pop_treat,age_treat,rent_treat,income_treat,25_treat,male_treat,black_treat,bachelor_treat,budget_treat,2015_treat,2016_treat,2017_treat,2018_treat,mayor_treat,manager_treat,dem_treat,rep_treat,time_treat
0,2015-08,32,Albuquerque city,NM,NaN,0,2012-01-01 00:00:00,1,2015,7,1,0,0,0,Albuquerque cityNM,"Albuquerque city, New Mexico",556859.0,22946.0,20933.0,22234.0,21153.0,271069.0,401281.0,18239.0,266937.0,36.0,816.0,26910.0,69694.0,35.0,2000.0,2016.0,0.157106,0.486782,0.032753,0.720615,0.479362,0.125156,Albuquerque city,New Mexico,NM,Albuquerque city,NM,Solid Democratic,Mountain,Mayor-Council,"$526,400,000.00",Democratic,1,0,0,0,0,0,0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0
1,2015-09,149,Albuquerque city,NM,NaN,0,2012-01-01 00:00:00,1,2015,8,1,0,0,0,Albuquerque cityNM,"Albuquerque city, New Mexico",556859.0,22946.0,20933.0,22234.0,21153.0,271069.0,401281.0,18239.0,266937.0,36.0,816.0,26910.0,69694.0,35.0,2000.0,2016.0,0.157106,0.486782,0.032753,0.720615,0.479362,0.125156,Albuquerque city,New Mexico,NM,Albuquerque city,NM,Solid Democratic,Mountain,Mayor-Council,"$526,400,000.00",Democratic,1,0,0,0,0,0,0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0
2,2015-10,155,Albuquerque city,NM,NaN,0,2012-01-01 00:00:00,1,2015,9,1,0,0,0,Albuquerque cityNM,"Albuquerque city, New Mexico",556859.0,22946.0,20933.0,22234.0,21153.0,271069.0,401281.0,18239.0,266937.0,36.0,816.0,26910.0,69694.0,35.0,2000.0,2016.0,0.157106,0.486782,0.032753,0.720615,0.479362,0.125156,Albuquerque city,New Mexico,NM,Albuquerque city,NM,Solid Democratic,Mountain,Mayor-Council,"$526,400,000.00",Democratic,1,0,0,0,0,0,0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0
3,2015-11,226,Albuquerque city,NM,NaN,0,2012-01-01 00:00:00,1,2015,10,1,0,0,0,Albuquerque cityNM,"Albuquerque city, New Mexico",556859.0,22946.0,20933.0,22234.0,21153.0,271069.0,401281.0,18239.0,266937.0,36.0,816.0,26910.0,69694.0,35.0,2000.0,2016.0,0.157106,0.486782,0.032753,0.720615,0.479362,0.125156,Albuquerque city,New Mexico,NM,Albuquerque city,NM,Solid Democratic,Mountain,Mayor-Council,"$526,400,000.00",Democratic,1,0,0,0,0,0,0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0
4,2015-12,234,Albuquerque city,NM,NaN,0,2012-01-01 00:00:00,1,2015,11,1,0,0,0,Albuquerque cityNM,"Albuquerque city, New Mexico",556859.0,22946.0,20933.0,22234.0,21153.0,271069.0,401281.0,18239.0,266937.0,36.0,816.0,26910.0,69694.0,35.0,2000.0,2016.0,0.157106,0.486782,0.032753,0.720615,0.479362,0.125156,Albuquerque city,New Mexico,NM,Albuquerque city,NM,Solid Democratic,Mountain,Mayor-Council,"$526,400,000.00",Democratic,1,0,0,0,0,0,0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0
5,2016-01,248,Albuquerque city,NM,NaN,0,2012-01-01 00:00:00,1,2016,12,0,1,0,0,Albuquerque cityNM,"Albuquerque city, New Mexico",556859.0,22946.0,20933.0,22234.0,21153.0,271069.0,401281.0,18239.0,266937.0,36.0,816.0,26910.0,69694.0,35.0,2000.0,2016.0,0.157106,0.486782,0.032753,0.720615,0.479362,0.125156,Albuquerque city,New Mexico,NM,Albuquerque city,NM,Solid Democratic,Mountain,Mayor-Council,"$526,400,000.00",Democratic,1,0,0,0,0,0,0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0
6,2016-02,221,Albuquerque city,NM,NaN,0,2012-01-01 00:00:00,1,2016,13,0,1,0,0,Albuquerque cityNM,"Albuquerque city, New Mexico",556859.0,22946.0,20933.0,22234.0,21153.0,271069.0,401281.0,18239.0,266937.0,36.0,816.0,26910.0,69694.0,35.0,2000.0,2016.0,0.157106,0.486782,0.032753,0.720615,0.479362,0.125156,Albuquerque city,New Mexico,NM,Albuquerque city,NM,Solid Democratic,Mountain,Mayor-Council,"$526,400,000.00",Democratic,1,0,0,0,0,0,0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0
7,2016-03,262,Albuquerque city,NM,NaN,0